In [ ]:
#| default_exp configs

# Configs
> Several dictionaries used to generate `.toml` configuration files copied under `/home/.marisco` folder and associated utilities function. These `.toml` files can be then adapted to your specific needs if required.



In [ ]:
#| export
from pathlib import Path
import os
import re
from typing import Dict, Callable
import pandas as pd
import fastcore.all as fc
from netCDF4 import Dataset
from marisco.inout import read_toml

In [ ]:
#| exports
AVOGADRO = 6.02214076e23

In [ ]:
#| exports
NC_DIM = 'id'

:::{.callout-tip}

**FEEDBACK FOR NEXT VERSION**: `PH` was included in ``NC_VARS`` but not in ``field-definition.ipynb``.

:::

In [ ]:
#| exports
# Lookup variables names to MARIS NetCDF4 template variable names
NC_VARS = {
    'ID': 'id',
    'LON': 'lon',
    'LAT': 'lat',
    'SMP_DEPTH': 'smp_depth',
    'TOT_DEPTH': 'tot_depth',
    'TIME': 'time',
    'AREA': 'area',
    'SMP_ID': 'smp_id',
    'NUCLIDE': 'nuclide',
    'VALUE': 'value',
    'UNIT': 'unit',
    'UNC': 'unc',
    'DL': 'dl',
    'FILT': 'filt',
    'COUNT_MET': 'count_met',
    'SAMP_MET': 'samp_met',
    'PREP_MET': 'prep_met',
    'VOL': 'vol',
    'SAL': 'sal',
    'TEMP': 'temp',
    'PH': 'ph',
    'BIO_GROUP': 'bio_group',
    'SPECIES': 'species',
    'BODY_PART': 'body_part',
    'SED_TYPE': 'sed_type',
    'TOP': 'top',
    'BOTTOM': 'bottom',
    'DRYWT': 'drywt',
    'WETWT': 'wetwt',
    'LAB': 'lab'
    }

In [ ]:
#| exports
OR_VARS = {
    'LON': 'longitude', 
    'LAT': 'latitude',
    'SMP_DEPTH': 'sampdepth',
    'TOT_DEPTH': 'totdepth',
    'TIME': 'begperiod',
    'AREA': 'area',
    'NUCLIDE': 'nuclide_id',
    'VALUE': 'activity',
    'UNIT': 'unit_id',
    'UNC': 'uncertaint',
    'DL': 'detection',
    'FILT': 'filtered',
    'COUNT_MET': 'counmet_id',
    'SAMP_MET': 'sampmet_id', 
    'PREP_MET': 'prepmet_id',
    'VOL': 'volume',
    'SAL': 'salinity',
    'TEMP': 'temperatur',
    'SPECIES': 'species_id',
    'BODY_PART': 'bodypar_id',
    'SED_TYPE': 'sedtype_id',
    'TOP': 'sliceup',
    'BOTTOM': 'slicedown',
    'DRYWT': 'drywt',
    'WETWT': 'wetwt',
    'LAB': 'lab_id'
}

In [ ]:
#| exports
NC_GROUPS = {'BIOTA': 'biota',
             'SEAWATER': 'seawater',
             'SEDIMENT': 'sediment',
             'SUSPENDED_MATTER': 'suspended_matter'}

In [ ]:
#| exports
SMP_TYPE_LUT = {
    'SEAWATER': 1,
    'BIOTA': 2,
    'SEDIMENT': 3,
    'SUSPENDED_MATTER': 4
}

In [ ]:
#| exports
NC_DTYPES = {
    'AREA': {
        'name': 'area_t', 
        'fname': 'dbo_area.xlsx',
        'key': 'displayName', 
        'value':'areaId'
    },
    'BIO_GROUP': {
        'name': 'bio_group_t', 
        'fname': 'dbo_biogroup.xlsx',
        'key': 'biogroup', 
        'value':'biogroup_id'
    },
    'BODY_PART': {
        'name': 'body_part_t', 
        'fname': 'dbo_bodypar.xlsx',
        'key': 'bodypar', 
        'value':'bodypar_id'
    },
    'COUNT_MET': {
        'name': 'count_met_t', 
        'fname': 'dbo_counmet.xlsx',
        'key': 'counmet',
        'value':'counmet_id'
    },
    'DL': {
        'name': 'dl_t', 
        'fname': 'dbo_detectlimit.xlsx',
        'key': 'name_sanitized', 
        'value':'id'
    },
    'FILT': {
        'name': 'filt_t', 
        'fname': 'dbo_filtered.xlsx',
        'key': 'name',
        'value':'id'
    },
    'NUCLIDE': {
        'name': 'nuclide_t', 
        'fname': 'dbo_nuclide.xlsx',
        'key': 'nc_name',
        'value':'nuclide_id'
    },
    'PREP_MET': {
        'name': 'prep_met_t', 
        'fname': 'dbo_prepmet.xlsx', 
        'key': 'prepmet',
        'value':'prepmet_id'
    },
    'SAMP_MET': {
        'name': 'samp_met_t', 
        'fname': 'dbo_sampmet.xlsx', 
        'key': 'sampmet',
        'value':'sampmet_id'
    },
    'SED_TYPE': {
        'name': 'sed_type_t', 
        'fname': 'dbo_sedtype.xlsx', 
        'key': 'sedtype', 
        'value':'sedtype_id'
    },
    'SPECIES': {
        'name': 'species_t', 
        # 'fname': 'dbo_species_cleaned.xlsx',
        'fname': 'dbo_species_2024_11_19.xlsx',
        'key': 'species', 
        'value':'species_id'
    },
    'UNIT': {
        'name': 'unit_t', 
        'fname': 'dbo_unit.xlsx', 
        'key': 'unit_sanitized', 
        'value':'unit_id'
    },
    'LAB': {
        'name': 'lab_t', 
        #'fname': 'dbo_lab.xlsx', 
        'fname': 'dbo_lab_cleaned.xlsx', 
        'key': 'lab', 
        'value':'lab_id'
    }
}


In [ ]:
#| exports
CFG_FNAME = 'configs.toml'
#CDL_FNAME = 'cdl.toml'
NUCLIDE_LOOKUP_FNAME = 'dbo_nuclide.xlsx'
MARISCO_CFG_DIRNAME = '.marisco'

In [ ]:
#| exports
def base_path(): 
    "Return the path to the `.marisco` folder under your home directory."
    return Path.home() / MARISCO_CFG_DIRNAME

By default, we create a folder named `.marisco` under your home directory that will receive all configuration files as defined in `BASE_PATH`:

In [ ]:
base_path()

Path('/home/marisco/.marisco')

In [ ]:
#| exports
CONFIGS = {
    'gh': {
        'owner': 'franckalbinet',
        'repo': 'marisco'
    },
    'names': {  
        'nc_template': 'maris-template.nc'
    },
    'dirs': {
        'lut': str(base_path() / 'lut'), # Look-up tables
        'cache': str(base_path() / 'cache'), # Cache (e.f WoRMS species)
        'tmp': str(base_path() / 'tmp')
    },
    'paths': {
        'nc_template': 'nbs/files/nc',
        'luts': 'nbs/files/lut'
    },
    # TO BE REMOVED:Now read directly from `maris.cdl` file
    # 'units': {
    #     'time': 'seconds since 1970-01-01 00:00:00.0'
    # },
    'zotero': {
        'api_key': os.getenv('ZOTERO_API_KEY'),
        'lib_id': '2432820'
    }
}

The `CONFIGS` dictionary defines general settings:

| key     | Value  | Description                                   |
|---------|-------------|--------------------------------------------|
| `dirs/lut`   | `/Users/franckalbinet/.marisco/lut`         | Location & name of the directory receiving lookup tables.          |
| `dirs/cache`   | `/Users/franckalbinet/.marisco/cache`         | Location & name of the directory receiving cache files such as WoRMS species retrieved.          |
| `dirs/tmp`    |  `/Users/franckalbinet/.marisco/tmp`         | Location & name of temporary files.       |
| `gh/owner`     |  `franckalbinet`       | GitHub account owner.                             |
| `gh/repo`      |  `marisco`       |   GitHub user used to download specific files (e.g lookup tables) during installation.                                  |
| `names/nc_template`   | `maris-template.nc` |  Name of the MARIS NetCDF4 template.                         |
| `paths/nc`   |  `nbs/files/nc`        |  GitHub repository directory name containing the MARIS NetCDF4 template.                             |
| `paths/luts`   |  `nbs/files/lut`        |  GitHub repository directory name containing lookup tables.                             |
| `zotero/api_key`  | `your-zotero-api-key`      |   Zotero API key ("ZOTERO_API_KEY" environment variable).                    |
| `zotero/lib_id`  |  `2432820`       | Zotero library ID.                                  |


The main `CONFIGS_CDL` dictionary, used to generate a [NetCDF CDL (Common Data Language)](https://www.unidata.ucar.edu/software/netcdf/workshops/2012/nc3model/Cdl.html) `.toml` file. This file is then used to generate a template MARIS netcdf file. For further details refers to the [`configs.ipynb`](https://github.com/franckalbinet/marisco/blob/main/nbs/api/configs.ipynb) file.

Below, the vars/defaults section printed:

In [ ]:
#| exports
def cfg(): 
    "Return the configuration as a dictionary."
    return read_toml(base_path() / CFG_FNAME)

In [ ]:
#| exports
def lut_path(): 
    "Return the path to the lookup tables directory."
    return Path(cfg()['dirs']['lut'])

In [ ]:
#| exports
def cache_path(): 
    "Return the path to the cache directory."
    return Path(cfg()['dirs']['cache'])

In [ ]:
#| exports

# TO BE REMOVED: Now read directly from `maris.cdl` file
CONFIGS_CDL = { 
    'placeholder': '_to_be_filled_in_',
    'grps': {
        'sea': {
            'name': 'seawater',
            'id': 1
        },
        'bio': {
            'name': 'biota',
            'id': 2
        },
        'sed': {
            'name': 'sediment',
            'id': 3
        },
        'sus': {
            'name': 'suspended-matter',
            'id': 4
        }
    },
    'global_attrs': {
        # Do not update keys. Only values if required
        'id': '', # zotero?
        'title': '',
        'summary': '',
        'keywords': '',
        'keywords_vocabulary': 'GCMD Science Keywords',
        'keywords_vocabulary_url': 'https://gcmd.earthdata.nasa.gov/static/kms/',
        'record': '',
        'featureType': '',
        'cdm_data_type': '',

        # Conventions
        'Conventions': 'CF-1.10 ACDD-1.3',

        # Publisher [ACDD1.3]
        'publisher_name': 'Paul MCGINNITY, Iolanda OSVATH, Florence DESCROIX-COMANDUCCI',
        'publisher_email': 'p.mc-ginnity@iaea.org, i.osvath@iaea.org, F.Descroix-Comanducci@iaea.org', 
        'publisher_url': 'https://maris.iaea.org',
        'publisher_institution': 'International Atomic Energy Agency - IAEA', 

        # Creator info [ACDD1.3]
        'creator_name': '',
        'institution': '',
        'metadata_link': '',
        'creator_email': '',
        'creator_url': '',
        'references': '',
        'license': ' '.join(['Without prejudice to the applicable Terms and Conditions', 
                             '(https://nucleus.iaea.org/Pages/Others/Disclaimer.aspx),',
                             'I hereby agree that any use of the data will contain appropriate',
                             'acknowledgement of the data source(s) and the IAEA Marine',
                             'Radioactivity Information System (MARIS).']),
        'comment': '',
        # Dataset info & coordinates [ACDD1.3]
        #'project': '', # Network long name
        #'platform': '', # Should be a long / full name
        'geospatial_lat_min': '', 
        'geospatial_lon_min': '',
        'geospatial_lat_max': '',
        'geospatial_lon_max': '',
        'geospatial_vertical_min': '',
        'geospatial_vertical_max': '',
        'geospatial_bounds': '', # wkt representation
        'geospatial_bounds_crs': 'EPSG:4326',

        # Time information
        'time_coverage_start': '',
        'time_coverage_end': '',
        #'time_coverage_resolution': '',
        'local_time_zone': '',
        'date_created': '',
        'date_modified': '',
        #
        # -- Additional metadata (custom to MARIS)
        #
        'publisher_postprocess_logs': ''
        },
    'dim': {
        'name': 'sample',
        'attrs': {
            'long_name': 'Sample ID of measurement'
        },
        'dtype': 'u8'
    },
    'vars': {    
        'defaults': {
            'lon': {
                'name': 'lon',
                'attrs': {
                    'long_name': 'Measurement longitude',
                    'standard_name': 'longitude',
                    'units': 'degrees_north',
                    'axis': 'Y',
                    '_CoordinateAxisType': 'Lon'
                },
                'dtype': 'f4'
            },
            'lat': {
                'name': 'lat',
                'attrs': {
                    'long_name': 'Measurement latitude',
                    'standard_name': 'latitude',
                    'units': 'degrees_east',
                    'axis': 'X',
                    '_CoordinateAxisType': 'Lat'
                },
                'dtype': 'f4'
            },
            'smp_depth': {
                'name': 'smp_depth',
                'attrs': {
                    'long_name': 'Sample depth below seal level',
                    'standard_name': 'sample_depth_below_sea_floor',
                    'units': 'm',
                    'axis': 'Z'
                },
                'dtype': 'f4'
            },
            'tot_depth': {
                'name': 'tot_depth',
                'attrs': {
                    'long_name': 'Total depth below seal level',
                    'standard_name': 'total_depth_below_sea_floor',
                    'units': 'm',
                    'axis': 'Z'
                },
                'dtype': 'f4'
            },
            'time': {
                'name': 'time',
                'attrs': {
                    'long_name': 'Time of measurement',
                    'standard_name': 'time',
                    'units': 'seconds since 1970-01-01 00:00:00.0',
                    'time_origin': '1970-01-01 00:00:00',
                    'time_zone': 'UTC',
                    'abbreviation': 'Date/Time',
                    'axis': 'T',
                    'calendar': 'gregorian'
                },
                'dtype': 'u8',
            },
            'area': {
                'name': 'area',
                'attrs': {
                    'long_name': 'Marine area/region id',
                    'standard_name': 'area_id'
                },
                'dtype': 'area_t'
            },
        },
        'bio': {
            'bio_group': {
                'name': 'bio_group',
                'attrs': {
                    'long_name': 'Biota group',
                    'standard_name': 'biota_group_tbd'
                },
                'dtype': 'bio_group_t'
            },
            'species': {
                'name': 'species',
                'attrs': {  
                    'long_name': 'Species',
                    'standard_name': 'species'
                },
                'dtype': 'species_t'
            },
            'body_part': {
                'name': 'body_part',
                'attrs': {
                    'long_name': 'Body part',
                    'standard_name': 'body_part_tbd'
                },
                'dtype': 'body_part_t' 
            }
        },
        'sed': {
            'sed_type': {
                'name': 'sed_type',
                'attrs': {
                    'long_name': 'Sediment type',
                    'standard_name': 'sediment_type_tbd'
                },
                'dtype': 'sed_type_t'
            },
            'top': {
                'name': 'top',
                'attrs': {
                    'long_name': 'Top depth of sediment layer',
                    'standard_name': 'top_depth_of_sediment_layer_tbd'
                },
                'dtype': 'f4'
            },
            'bottom': {
                'name': 'bottom',
                'attrs': {
                    'long_name': 'Bottom depth of sediment layer',
                    'standard_name': 'bottom_depth_of_sediment_layer_tbd'
                },
                'dtype': 'f4'
            },
        },
        'suffixes':  {
            'uncertainty': {
                'name': '_unc',
                'attrs': {
                    'long_name': ' uncertainty',
                    'standard_name': '_uncertainty'
                },
                'dtype': 'f4'
            },
            'detection_limit': {
                'name': '_dl',
                'attrs': {
                    'long_name': ' detection limit',
                    'standard_name': '_detection_limit'
                },
                'dtype': 'dl_t'
            },
            'volume': {
                'name': '_vol',
                'attrs': {
                    'long_name': ' volume',
                    'standard_name': '_volume'
                },
                'dtype': 'f4'
            },
            'salinity': {
                'name': '_sal',
                'attrs': {
                    'long_name': ' salinity',
                    'standard_name': '_sal'
                },
                'dtype': 'f4'
            },
            'temperature': {
                'name': '_temp',
                'attrs': {
                    'long_name': ' temperature',
                    'standard_name': '_temp'
                },
                'dtype': 'f4'
            },
            'filtered': {
                'name': '_filt',
                'attrs': {
                    'long_name': ' filtered',
                    'standard_name': '_filtered'
                },
                'dtype': 'filt_t'
            },
            'counting_method': {
                'name': '_counmet',
                'attrs': {
                    'long_name': ' counting method',
                    'standard_name': '_counting_method'
                },
                'dtype': 'counmet_t'
            },
            'sampling_method': {
                'name': '_sampmet',
                'attrs': {
                    'long_name': ' sampling method',
                    'standard_name': '_sampling_method'
                },
                'dtype': 'sampmet_t'
            },
            'preparation_method': {
                'name': '_prepmet',
                'attrs': {
                    'long_name': ' preparation method',
                    'standard_name': '_preparation_method'
                },
                'dtype': 'prepmet_t'
            },
            'unit': {
                'name': '_unit',
                'attrs': {
                    'long_name': ' unit',
                    'standard_name': '_unit'
                },
                'dtype': 'unit_t'
            }
        }
    },
    'enums': [
        {
            'name': 'area_t', 
            'fname': 'dbo_area.xlsx', 
            'key': 'displayName', 
            'value':'areaId'
        },
        {
            'name': 'bio_group_t', 
            'fname': 'dbo_biogroup.xlsx', 
            'key': 'biogroup', 
            'value':'biogroup_id'
        },
        {
            'name': 'body_part_t', 
            'fname': 'dbo_bodypar.xlsx', 
            'key': 'bodypar', 
            'value':'bodypar_id'
        },
        {
            'name': 'species_t', 
            'fname': 'dbo_species_cleaned.xlsx', 
            'key': 'species', 
            'value':'species_id'
        },
        {
            'name': 'sed_type_t', 
            'fname': 'dbo_sedtype.xlsx', 
            'key': 'sedtype', 
            'value':'sedtype_id'
        },  
        {
            'name': 'unit_t', 
            'fname': 'dbo_unit.xlsx', 
            'key': 'unit_sanitized', 
            'value':'unit_id'
        },
        {
            'name': 'dl_t', 
            'fname': 'dbo_detectlimit.xlsx', 
            'key': 'name_sanitized', 
            'value':'id'
        },
        {
            'name': 'filt_t', 
            'fname': 'dbo_filtered.xlsx', 
            'key': 'name',
            'value':'id'
        },
        {
            'name': 'counmet_t', 
            'fname': 'dbo_counmet.xlsx', 
            'key': 'counmet',
            'value':'counmet_id'
        },
        {
            'name': 'sampmet_t', 
            'fname': 'dbo_sampmet.xlsx', 
            'key': 'sampmet',
            'value':'sampmet_id'
        },
        {
            'name': 'prepmet_t', 
            'fname': 'dbo_prepmet.xlsx', 
            'key': 'prepmet',
            'value':'prepmet_id'
        }
        ]
}

In [ ]:
# fc.AttrDict(CONFIGS_CDL['vars']['defaults'])

In [ ]:
#|eval: false
# write_toml(Path('./files') / CDL_FNAME, CONFIGS_CDL)

In [ ]:
#| exports
# def cdl_cfg():
#     "Return the CDL (Common Data Language) configuration as a dictionary."
#     try:
#         return read_toml(base_path() / CDL_FNAME)
#     except FileNotFoundError:
#         return CONFIGS_CDL

In [ ]:
# | exports
# def grp_names(): 
#     "Return the group names as defined in `cdl.toml`."
#     return [v['name'] for v in cdl_cfg()['grps'].values()]

## Lookup tables paths

In [ ]:
#| exports
def nuc_lut_path(): 
    "Return the path to the nuclide lookup table."
    return Path(cfg()['dirs']['lut']) / NUCLIDE_LOOKUP_FNAME

In [ ]:
#| exports
def species_lut_path():
    "Return the path to the species lookup table."
    src_dir = lut_path()
    fname = NC_DTYPES['SPECIES']['fname']
    
    return src_dir / fname

In [ ]:
#| eval: false
species_lut_path()

Path('/home/marisco/.marisco/lut/dbo_species_2024_11_19.xlsx')

In [ ]:
#| exports
def bodyparts_lut_path():
    "Return the path to the body parts lookup table."
    src_dir = lut_path()
    fname = NC_DTYPES['BODY_PART']['fname']
    return src_dir / fname

In [ ]:
#| exports
def lab_lut_path():
    "Return the path to the laboratory lookup table."
    src_dir = lut_path()
    fname = NC_DTYPES['LAB']['fname']
    return src_dir / fname

In [ ]:
#| eval: false
lab_lut_path()

Path('/home/marisco/.marisco/lut/dbo_lab_cleaned.xlsx')

In [ ]:
#| exports
def biogroup_lut_path():
    "Return the path to the biota group lookup table."
    src_dir = lut_path()
    fname = NC_DTYPES['BIO_GROUP']['fname']
    return src_dir / fname

In [ ]:
#| eval: false
biogroup_lut_path()

Path('/home/marisco/.marisco/lut/dbo_biogroup.xlsx')

In [ ]:
#| exports
def sediments_lut_path():
    "Return the path to the sediment type lookup table."
    src_dir = lut_path()
    fname = NC_DTYPES['SED_TYPE']['fname']
    return src_dir / fname

In [ ]:
#| eval: false
sediments_lut_path()

Path('/home/marisco/.marisco/lut/dbo_sedtype.xlsx')

In [ ]:
#| exports
def unit_lut_path():
    "Return the path to the unit lookup table."
    src_dir = lut_path()
    fname = NC_DTYPES['UNIT']['fname']
    return src_dir / fname

In [ ]:
#| eval: false
unit_lut_path()

Path('/home/marisco/.marisco/lut/dbo_unit.xlsx')

In [ ]:
#| exports
def detection_limit_lut_path():
    "Return the path to the detection limit lookup table."
    src_dir = lut_path()
    fname = NC_DTYPES['DL']['fname']
    return src_dir / fname

In [ ]:
#| eval: false
detection_limit_lut_path()

Path('/home/marisco/.marisco/lut/dbo_detectlimit.xlsx')

In [ ]:
#| exports
def filtered_lut_path():
    "Return the path to the filtered lookup table."
    src_dir = lut_path()
    fname = NC_DTYPES['FILT']['fname']
    return src_dir / fname

In [ ]:
#| eval: false
filtered_lut_path()

Path('/home/marisco/.marisco/lut/dbo_filtered.xlsx')

In [ ]:
#| exports
def area_lut_path():
    "Return the path to the area lookup table."
    src_dir = lut_path()
    fname = NC_DTYPES['AREA']['fname']
    return src_dir / fname

# area_lut_path()

In [ ]:
#| eval: false
area_lut_path()

Path('/home/marisco/.marisco/lut/dbo_area.xlsx')

In [ ]:
#| exports
def prepmet_lut_path():
    "Return the path to the prepmet lookup table."
    src_dir = lut_path()
    fname = NC_DTYPES['PREP_MET']['fname']
    return src_dir / fname

# prepmet_lut_path()

In [ ]:
#| eval: false
prepmet_lut_path()

Path('/home/marisco/.marisco/lut/dbo_prepmet.xlsx')

In [ ]:
#| exports
def sampmet_lut_path():
    "Return the path to the sampmet lookup table."
    src_dir = lut_path()
    fname = NC_DTYPES['SAMP_MET']['fname']
    return src_dir / fname

# sampmet_lut_path()

In [ ]:
#| eval: false
sampmet_lut_path()

Path('/home/marisco/.marisco/lut/dbo_sampmet.xlsx')

In [ ]:
#| exports
def counmet_lut_path():
    "Return the path to the counmet lookup table."
    src_dir = lut_path()
    fname = NC_DTYPES['COUNT_MET']['fname']
    return src_dir / fname

# counmet_lut_path()

In [ ]:
#| eval: false
counmet_lut_path()

Path('/home/marisco/.marisco/lut/dbo_counmet.xlsx')

## Utilities function

In [ ]:
#| exports
NETCDF_TO_PYTHON_TYPE = {
    'u8': int,
    'f4': float
    }

In [ ]:
#| exports
# def name2grp(
#     name: str, # Group name
#     cdl: dict, # CDL configuration
#     ):
#     # Reverse `cdl.toml` config group dict so that group config key can be retrieve based on its name
#     return {v['name']:k  for k, v in cdl['grps'].items()}[name]

Example:

In [ ]:
#|eval: false
# TO BE REMOVED: Now read directly from `maris.cdl` file
# name2grp('seawater', cdl=CONFIGS_CDL)

In [ ]:
#| exports
def nc_tpl_name():
    "Return the name of the MARIS NetCDF template as defined in `configs.toml`"
    p = base_path()
    return read_toml(p / 'configs.toml')['names']['nc_template']

In [ ]:
#| exports
def nc_tpl_path():
    "Return the path of the MARIS NetCDF template as defined in `configs.toml`"
    p = base_path()
    return p / read_toml(p / 'configs.toml')['names']['nc_template']

In [ ]:
#| exports
def get_time_units(
    nc_path: Callable=nc_tpl_path # Function returning Path to NetCDF template file
    ) -> str: # Time units string (e.g. 'seconds since 1970-01-01 00:00:00.0')
    "Get the units attribute of the time variable from a NetCDF file."
    with Dataset(nc_tpl_path(), 'r') as nc:
        for group in nc.groups.values():
            if 'time' in group.variables:
                return group.variables['time'].units
                
    raise ValueError("Time variable not found in NetCDF file")

Usage example:

In [ ]:
#| eval: false
time_units = get_time_units(); time_units

'seconds since 1970-01-01 00:00:00.0'

## Enumeration types

Enumeration types are used to avoid using strings as NetCDF4 variable values. Instead, enumeration types (lookup tables) such as `{'Crustaceans': 2, 'Echinoderms': 3, ...}` are prepended to the NetCDF file template and associated ids (integers) are used as values.

In [ ]:
#| exports
def sanitize(
    s: str|float # String or float to sanitize
    ) -> str|float:  # Sanitized string or original float
    """
    Sanitize dictionary key to comply with NetCDF enumeration type:
    
    - Remove `(`, `)`, `.`, `/`, `-`
    - Strip the string
    - Return original value if it's not a string (e.g., NaN)
    """
    if isinstance(s, str):
        s = re.sub(r'[().]', '', s)
        return re.sub(r'[/-]', ' ', s).strip()
    elif pd.isna(s):  # This covers np.nan, None, and pandas NaT
        return s
    else:
        return str(s).strip()

For example:

In [ ]:
fc.test_eq(sanitize('key (sanitized)'), 'key sanitized')
fc.test_eq(sanitize('key san.itized'), 'key sanitized')
fc.test_eq(sanitize('key-sanitized'), 'key sanitized')
fc.test_eq(sanitize('key/sanitized'), 'key sanitized')

NetCDF4 enumeration type seems to not accept keys containing non alphanumeric characters like parentheses, dots, slash, ... As a result, MARIS lookup table needs to be sanitized.

In [ ]:
#| exporti
def try_int(x):
    "Try to convert `x` to an integer."
    try:
        return int(x)
    except (ValueError, TypeError):
        return x

In [ ]:
#| exports
def get_lut(
    src_dir: str, # Directory containing lookup tables
    fname: str, # Excel file lookup table name
    key: str, # Excel file column name to be used as dict keys 
    value: str, # Excel file column name to be used as dict values 
    do_sanitize: bool=True, # Sanitization required?
    reverse: bool=False, # Reverse lookup table (value, key)
    check_duplicates: bool=False # Check for duplicates in lookup table
    ) -> Dict[str, int]: # MARIS lookup table (key, value)
    "Convert MARIS db lookup table excel file to dictionary `{'name': id, ...}` or `{id: name, ...}` if `reverse` is True."
    fname = Path(src_dir) / fname
    df = pd.read_excel(fname, usecols=[key, value]).dropna(subset=value)
    
    if check_duplicates:
        duplicates = df[key][df[key].duplicated()].tolist()
        if duplicates: print(f"Warning: {fname.name}: found duplicate keys: {duplicates}")
        
    df[value] = df[value].astype('int')
    df = df.set_index(key)
    lut = df[value].to_dict()
    if do_sanitize: lut = {sanitize(k): v for k, v in lut.items()}
    lut = {try_int(k): try_int(v) for k, v in lut.items()}    
    return {v: k for k, v in lut.items()} if reverse else lut

For example:

In [ ]:
#|eval: false
lut_src_dir = './files/lut'
get_lut(lut_src_dir, 'dbo_biogroup.xlsx', key='biogroup', value='biogroup_id', reverse=False)

{'Not applicable': -1,
 'Not available': 0,
 'Birds': 1,
 'Crustaceans': 2,
 'Echinoderms': 3,
 'Fish': 4,
 'Mammals': 5,
 'Molluscs': 6,
 'Others': 7,
 'Plankton': 8,
 'Polychaete worms': 9,
 'Reptile': 10,
 'Seaweeds and plants': 11,
 'Cephalopods': 12,
 'Gastropods': 13,
 'Bivalves': 14}

In [ ]:
#| exports
class Enums():
    "Return dictionaries of MARIS NetCDF's enumeration types."
    def __init__(self, 
                 lut_src_dir: str, # Directory containing lookup tables
                 dtypes: Dict[str, Dict[str, str]]=NC_DTYPES, # Custom NetCDF types
                 ):
        fc.store_attr()
        self.types = self.lookup()
        
    def filter(self, var_name, values):
        return {name: id for name, id in self.types[var_name].items() if id in values}
    
    def lookup(self):
        types = {}
        for var_name, dtype in self.dtypes.items():
            name, fname, key, value = dtype.values()
            lut = get_lut(self.lut_src_dir, fname, key=key, value=value)
            types[var_name] = lut
        return types

In [ ]:
#|eval: false
lut_src_dir_test = './files/lut'
enums = Enums(lut_src_dir=lut_src_dir_test)

In [ ]:
#|eval: false
enums.types['DL']

{'Not applicable': -1,
 'Not available': 0,
 'Detected value': 1,
 'Detection limit': 2,
 'Not detected': 3,
 'Derived': 4}

In [ ]:
#|eval: false
for k,v in enums.types['SPECIES'].items():
    print(k, v)

NOT AVAILABLE 0
Aristeus antennatus 1
Apostichopus 2
Saccharina japonica var religiosa 3
Siganus fuscescens 4
Alpheus dentipes 5
Hexagrammos agrammus 6
Ditrema temminckii 7
Parapristipoma trilineatum 8
Scombrops boops 9
Pseudopleuronectes schrenki 10
Desmarestia ligulata 11
Saccharina japonica 12
Neodilsea yendoana 13
Costaria costata 14
Sargassum yezoense 15
Acanthephyra pelagica 16
Sargassum ringgoldianum 17
Acanthephyra quadrispinosa 18
Sargassum thunbergii 19
Sargassum patens 20
Asterias rubens 21
Sargassum miyabei 22
Homarus gammarus 23
Acanthephyra stylorostratis 24
Acanthocybium solandri 25
Acanthopagrus bifasciatus 26
Acanthophora muscoides 27
Acanthophora spicifera 28
Acanthurus triostegus 29
Actinopterygii 30
Adamussium colbecki 31
Ahnfeltiopsis densa 32
Alepes melanoptera 33
Ampharetidae 34
Anchoviella lepidentostole 35
Anguillidae 36
Aphroditidae 37
Arnoglossus 38
Aurigequula fasciata 39
Balaenoptera musculus 40
Balaenoptera physalus 41
Balistes 42
Beryciformes 43
Bryopsis 